In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import torchtext
from torchtext import vocab, data
from torchtext.datasets import language_modeling

from fastai.rnn_reg import *
from fastai.rnn_train import *
from fastai.nlp import *
from fastai.lm_rnn import *
from fastai.structured import *
from fastai import sgdr

import spacy
spacy_en = spacy.load('en')

import dill as pickle

# pandas and plotting config
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_colwidth', -1)

In [4]:
from tqdm import tqdm as tqdm_cls
inst = tqdm_cls._instances
for i in range(len(inst)): inst.pop().close()

AttributeError: type object 'tqdm' has no attribute '_instances'

In [19]:
train = pd.read_csv("C://Users/hafid/Dropbox/3.SelfStudy/kaggle/ai6-kaggle-toxic/data/train/train.csv")
test = pd.read_csv("C://Users/hafid/Dropbox/3.SelfStudy/kaggle/ai6-kaggle-toxic/data/test/test.csv")
# subm = pd.read_csv("C://Users/hafid\Dropbox/3.SelfStudy/kaggle/jigsaw-toxic-comment-classification/sample_submission.csv")

In [26]:
PATH='C://Users/hafid/Dropbox/3.SelfStudy/kaggle/ai6-kaggle-toxic/'

TRN_PATH = 'data/train/'
VAL_PATH = 'data/test/'
TRN = f'{PATH}{TRN_PATH}'
VAL = f'{PATH}{VAL_PATH}'

os.listdir(PATH)

train = pd.read_csv(f'{TRN}train.csv')
test = pd.read_csv(f'{VAL}test.csv')


In [11]:
print(f'Train size: {len(train)} | Test size: {len(test)}')
display(train.head(3))
display(test.head(3))
# display(sample_subm_df.head(3))

Train size: 159571 | Test size: 153164


,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,"Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27",0,0,0,0,0,0
1,000103f0d9cfb60f,"D'aww! He matches this background colour I'm seemingly stuck with. Thanks. (talk) 21:51, January 11, 2016 (UTC)",0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It's just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page. He seems to care more about the formatting than the actual info.",0,0,0,0,0,0


,id,comment_text
0,00001cee341fdb12,"Yo bitch Ja Rule is more succesful then you'll ever be whats up with you and hating you sad mofuckas...i should bitch slap ur pethedic white faces and get you to kiss my ass you guys sicken me. Ja rule is about pride in da music man. dont diss that shit on him. and nothin is wrong bein like tupac he was a brother too...fuckin white boys get things right next time.,"
1,0000247867823ef7,"== From RfC == \n\n The title is fine as it is, IMO."
2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lapland — / """


We take a look at the overall statistics of the data. Also included another label named 'none' to represent comments that have no negative sentiments.

In [29]:
label_cols = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
train['none'] = 1 - train[label_cols].max(axis=1)

DataFrameSummary(train).summary()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,none
count,NaN,NaN,159571,159571,159571,159571,159571,159571,159571
mean,NaN,NaN,0.0958445,0.00999555,0.0529482,0.00299553,0.0493636,0.00880486,0.898321
std,NaN,NaN,0.294379,0.0994771,0.223931,0.0546496,0.216627,0.0934205,0.302226
min,NaN,NaN,0,0,0,0,0,0,0
25%,NaN,NaN,0,0,0,0,0,0,1
50%,NaN,NaN,0,0,0,0,0,0,1
75%,NaN,NaN,0,0,0,0,0,0,1
max,NaN,NaN,1,1,1,1,1,1,1
counts,159571,159571,159571,159571,159571,159571,159571,159571,159571
uniques,159571,159571,2,2,2,2,2,2,2


### Data Cleaning

In [12]:
#https://drive.google.com/file/d/0B1yuv8YaUVlZZ1RzMFJmc1ZsQmM/view
# Aphost lookup dict
APPO = {
"aren't" : "are not",
"can't" : "cannot",
"couldn't" : "could not",
"didn't" : "did not",
"doesn't" : "does not",
"don't" : "do not",
"hadn't" : "had not",
"hasn't" : "has not",
"haven't" : "have not",
"he'd" : "he would",
"he'll" : "he will",
"he's" : "he is",
"i'd" : "I would",
"i'd" : "I had",
"i'll" : "I will",
"i'm" : "I am",
"isn't" : "is not",
"it's" : "it is",
"it'll":"it will",
"i've" : "I have",
"let's" : "let us",
"mightn't" : "might not",
"mustn't" : "must not",
"shan't" : "shall not",
"she'd" : "she would",
"she'll" : "she will",
"she's" : "she is",
"shouldn't" : "should not",
"that's" : "that is",
"there's" : "there is",
"they'd" : "they would",
"they'll" : "they will",
"they're" : "they are",
"they've" : "they have",
"we'd" : "we would",
"we're" : "we are",
"weren't" : "were not",
"we've" : "we have",
"what'll" : "what will",
"what're" : "what are",
"what's" : "what is",
"what've" : "what have",
"where's" : "where is",
"who'd" : "who would",
"who'll" : "who will",
"who're" : "who are",
"who's" : "who is",
"who've" : "who have",
"won't" : "will not",
"wouldn't" : "would not",
"you'd" : "you would",
"you'll" : "you will",
"you're" : "you are",
"you've" : "you have",
"'re": " are",
"wasn't": "was not",
"we'll":" will",
"didn't": "did not",
"tryin'":"trying"
}

In [14]:
!pip install nltk

  Running setup.py bdist_wheel for nltk: started
  Running setup.py bdist_wheel for nltk: finished with status 'done'
  Stored in directory: C:\Users\hafid\AppData\Local\pip\Cache\wheels\18\9c\1f\276bc3f421614062468cb1c9d695e6086d0c73d67ea363c501
Successfully built nltk


In [15]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
from nltk.stem.wordnet import WordNetLemmatizer

eng_stopwords = set(stopwords.words("english"))
lem = WordNetLemmatizer()
tweet_tokenizer=TweetTokenizer()

def clean(comment):
    """
    This function receives comments and returns clean word-list
    """
    #Convert to lower case , so that Hi and hi are the same
    comment=comment.lower()
    #remove \n
    comment=re.sub("\\n","",comment)
    # remove leaky elements like ip,user
    comment=re.sub("\d{1,3}.\d{1,3}.\d{1,3}.\d{1,3}","",comment)
    #removing usernames
    comment=re.sub("\[\[.*\]","",comment)
    
    #Split the sentences into words
    words=tweet_tokenizer.tokenize(comment)
    
    # (')aphostophe  replacement (ie)   you're --> you are  
    # ( basic dictionary lookup : master dictionary present in a hidden block of code)
    words=[APPO[word] if word in APPO else word for word in words]
    words=[lem.lemmatize(word, "v") for word in words]
    words = [w for w in words if not w in eng_stopwords]
    
    clean_sent=" ".join(words)
    # remove any non alphanum,digit character
    #clean_sent=re.sub("\W+"," ",clean_sent)
    #clean_sent=re.sub("  "," ",clean_sent)
    return(clean_sent)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hafid\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\hafid\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.


In [16]:
train['clean_comment_text'] = train.comment_text.apply(lambda x: clean(x))
test['clean_comment_text'] = test.comment_text.apply(lambda x: clean(x))

### Build Dataset and ModelData Classes

In [17]:
class TextMultiLabelDataset(torchtext.data.Dataset):
    def __init__(self, df, tt_text_field, tt_label_field, txt_col, lbl_cols, **kwargs):
        # torchtext Field objects
        fields = [('text', tt_text_field)]
        for l in lbl_cols: fields.append((l, tt_label_field))
            
        is_test = False if lbl_cols[0] in df.columns else True
        n_labels = len(lbl_cols)
        
        examples = []
        for idx, row in df.iterrows():
            if not is_test:
                lbls = [ row[l] for l in lbl_cols ]
            else:
                lbls = [0.0] * n_labels
                
            txt = str(row[txt_col])
            examples.append(data.Example.fromlist([txt]+lbls, fields))
                            
        super().__init__(examples, fields, **kwargs)

    @staticmethod
    def sort_key(example): 
        return len(example.text)
    
    @classmethod
    def splits(cls, text_field, label_field, train_df, txt_col, lbl_cols, val_df=None, test_df=None, **kwargs):
        # build train, val, and test data
        train_data, val_data, test_data = (None, None, None)
        
        if train_df is not None: 
            train_data = cls(train_df.copy(), text_field, label_field, txt_col, lbl_cols, **kwargs)
        if val_df is not None: 
            val_data = cls(val_df.copy(), text_field, label_field, txt_col, lbl_cols, **kwargs)
        if test_df is not None: 
            test_data = cls(test_df.copy(), text_field, label_field, txt_col, lbl_cols, **kwargs)

        return tuple(d for d in (train_data, val_data, test_data) if d is not None)

In [18]:
class TextMultiLabelDataLoader():
    def __init__(self, src, x_fld, y_flds, y_dtype='torch.cuda.FloatTensor'):
        self.src, self.x_fld, self.y_flds = src, x_fld, y_flds
        self.y_dtype = y_dtype

    def __len__(self): return len(self.src)#-1

    def __iter__(self):
        it = iter(self.src)
        for i in range(len(self)):
            b = next(it)
            
            if (len(self.y_flds) > 1):
                targ = [ getattr(b, y) for y in self.y_flds ] 
                targ = torch.stack(targ, dim=1).type(self.y_dtype)
            else: 
                targ = getattr(b, self.y_flds[0])
                targ = targ.type(self.y_dtype)

            yield getattr(b, self.x_fld), targ

class TextMultiLabelData(ModelData):

    @classmethod
    def from_splits(cls, path, splits, bs, text_name='text', label_names=['label'], 
                    target_dtype='torch.cuda.FloatTensor'):
        
        text_fld = splits[0].fields[text_name]
        
        label_flds = []
        if (len(label_names) == 1): 
            label_fld = splits[0].fields[label_names[0]]
            label_flds.append(label_fld)
            if (label_fld.use_vocab): 
                label_fld.build_vocab(splits[0])
                target_dtype = 'torch.cuda.LongTensor'
        else:
            for n in label_names:
                label_fld = splits[0].fields[n]
                label_flds.append(label_fld)

        iters = torchtext.data.BucketIterator.splits(splits, batch_size=bs)
        trn_iter,val_iter,test_iter = iters[0],iters[1],None
        test_dl = None
        if len(iters) == 3:
            test_iter = iters[2]
            test_dl = TextMultiLabelDataLoader(test_iter, text_name, label_names, target_dtype)
        trn_dl = TextMultiLabelDataLoader(trn_iter, text_name, label_names, target_dtype)
        val_dl = TextMultiLabelDataLoader(val_iter, text_name, label_names, target_dtype)

        obj = cls.from_dls(path, trn_dl, val_dl, test_dl)
        obj.bs = bs
        obj.pad_idx = text_fld.vocab.stoi[text_fld.pad_token]
        obj.nt = len(text_fld.vocab)

        # if multiple labels, assume the # of classes = the # of labels 
        if (len(label_names) > 1):
            c = len(label_names)
        # if label has a vocab, assume the vocab represents the # of classes
        elif (hasattr(label_flds[0], 'vocab')): 
            c = len(label_flds[0].vocab)
        else:
            c = 1
            
        obj.c = c

        return obj

### Define tokenizer

In [19]:
import re, string
re_tok = re.compile(f'([{string.punctuation}“”¨«»®´·º½¾¿¡§£₤‘’])')
def tokenizer(s): return re_tok.sub(r' \1 ', s).split()

In [20]:
bsz = 8 #16 #32 #64

max_tokens = 20000 # max number of words based on frequency
max_len = None #100      # max length of each comment to look at

n_hidden = 256
n_fac = 64

### Define classifier

In [21]:
class LstmClassifier(nn.Module):
    def __init__(self, vocab_size, n_fac, bsz, 
                 fc_szs=[], fc_drops=[], n_lstm_hidden=256, n_lstm_layers=1, out_sz=1,
                 lstm_drop=0.5, is_multi=False, y_range=None, use_bn=False):
        
        super().__init__()
        
        self.vocab_size, self.out_sz = vocab_size, out_sz
        self.n_lstm_layers, self.n_lstm_hidden = n_lstm_layers, n_lstm_hidden
        
        self.is_multi = is_multi
        self.y_range = y_range
        self.use_bn = use_bn
        
        self.e = nn.Embedding(vocab_size, n_fac)
        self.e.weight.data.uniform_(-0.1, 0.1)
        
        self.rnn = nn.LSTM(n_fac, n_lstm_hidden, n_lstm_layers, dropout=lstm_drop)
        
        fc_szs = [n_lstm_hidden] + fc_szs
        
        self.linears, self.linear_drops, self.linear_bns = [], [], []
        if (len(fc_szs) > 1):
            self.linears = nn.ModuleList(
                [ nn.Linear(fc_szs[idx], sz) for idx, sz in enumerate(fc_szs[1:]) ]
            )
            
            self.linear_bns = nn.ModuleList(
                [ nn.BatchNorm1d(sz) for sz in fc_szs[1:] ]
            )
            
            for l in self.linears: kaiming_normal(l.weight.data)
                
        if (len(fc_drops) > 0):
            self.linear_drops = nn.ModuleList([ nn.Dropout(d) for d in fc_drops ])
                
        self.outp = nn.Linear(fc_szs[-1], out_sz)
        kaiming_normal(self.outp.weight.data)
            
        self.h = self.init_hidden(bsz)
        
    def forward(self, words):
        bsz = words[0].size(0)
        if (self.h[0].size(1) != bsz): self.h = self.init_hidden(bsz)
            
        x, h = self.rnn(self.e(words), self.h)
        self.h = repackage_var(h)
        
        x = x[-1]
        
        for l, d, b in zip(self.linears, self.linear_drops, self.linear_bns):
            x = F.relu(l(x))
            if (self.use_bn): x = b(x)
            x = d(x)
        
        x = self.outp(x)

        if (self.is_multi):
            return F.sigmoid(x)
        
        if (not self.is_multi and self.out_sz > 1):
            return F.log_softmax(x)
        
        if (self.y_range):
            x = F.sigmoid(x)
            x = x * (self.y_range[1] - self.y_range[0])
            x = x + self.y_range[0]
            
        return x
    
    def init_hidden(self, bsz):
        return(V(torch.zeros(self.n_lstm_layers, bsz, self.n_lstm_hidden)), 
               V(torch.zeros(self.n_lstm_layers, bsz, self.n_lstm_hidden)))

### Multi label classification
Predict class probabilities where multiple classes can be true

In [32]:
txt_col = 'comment_text'

val_idxs = get_cv_idxs(len(train), val_pct=0.001)

train_df =  train.drop(val_idxs)
val_df = train.copy().iloc[val_idxs]

len(train_df), len(val_df), len(test)

(159412, 159, 153164)

In [33]:
#Ensure target fields are the correct datatype
train_df[label_cols + ['none']] = train_df[label_cols + ['none']].astype(np.float32)
val_df[label_cols + ['none']] = val_df[label_cols + ['none']].astype(np.float32)

train_df.dtypes

id               object 
comment_text     object 
toxic            float32
severe_toxic     float32
obscene          float32
threat           float32
insult           float32
identity_hate    float32
none             float32
dtype: object

In [48]:
# Since our label is essentially a OHE vector, set use_vocab=False for the label's torchtext field
tt_TEXT = data.Field(sequential=True, tokenize=tokenizer, fix_length=max_len)
tt_LABEL = data.Field(sequential=False, use_vocab=False,tensor_type=torch.cuda.FloatTensor) # use_vocab=False if labels are numerical

In [49]:
splits = TextMultiLabelDataset.splits(tt_TEXT, tt_LABEL, train_df, 
                                      txt_col, label_cols, val_df, test)

In [50]:
# ------ multi-label problem ------
t = splits[0].examples[0]
t.toxic, t.insult, ' '.join(t.text)

(0.0,
 0.0,
 "Explanation Why the edits made under my username Hardcore Metallica Fan were reverted ? They weren ' t vandalisms , just closure on some GAs after I voted at New York Dolls FAC . And please don ' t remove the template from the talk page since I ' m retired now . 89 . 205 . 38 . 27")

In [51]:
tt_TEXT.build_vocab(splits[0], max_size=max_tokens)

In [52]:
md = TextMultiLabelData.from_splits(PATH, splits, bsz, text_name='text', label_names=label_cols)

len(md.trn_dl), md.nt, md.c

(19927, 20002, 6)

In [53]:
x,y = next(iter(md.val_dl))

In [54]:
y.size(), y[:5,:]

(torch.Size([8, 6]), Variable containing:
  0  0  0  0  0  0
  0  0  0  0  0  0
  1  0  1  0  1  1
  0  0  0  0  0  0
  0  0  0  0  0  0
 [torch.cuda.FloatTensor of size 5x6 (GPU 0)])

In [55]:
md.val_dl.it.close()

In [56]:
print(tt_TEXT.vocab.stoi[tt_TEXT.pad_token])
print(tt_TEXT.vocab.itos[:10])

1
['<unk>', '<pad>', '.', ',', 'the', '"', 'to', 'I', 'of', "'"]


In [57]:
print('most common words: ')
tt_TEXT.vocab.freqs.most_common(10)

most common words: 


[('.', 679644),
 (',', 474397),
 ('the', 448846),
 ('"', 392207),
 ('to', 291675),
 ('I', 224237),
 ('of', 221226),
 ("'", 219203),
 ('and', 212449),
 ('a', 203738)]

In [58]:
print('least common words: ')
tt_TEXT.vocab.freqs.most_common()[-10:]

least common words: 


[('Gag01001', 1),
 ('Aberration', 1),
 ('1053', 1),
 ('Hanumakonda', 1),
 ('956CE', 1),
 ('automakers', 1),
 ('Boastful', 1),
 ('Superlatives', 1),
 ('Classifying', 1),
 ('CIU', 1)]

## Train

In [ ]:
m = LstmClassifier(md.nt, n_fac, bsz, [512, 256], [0.4, 0.2],
                   n_hidden, n_lstm_layers=1, out_sz=md.c, is_multi=True, use_bn=True).cuda()

lo = LayerOptimizer(optim.Adam, m, 1e-3, 1e-4)

In [ ]:
fit(m, md, 1, lo.opt, F.binary_cross_entropy) 

HBox(children=(IntProgress(value=0, description='Epoch', max=1), HTML(value='')))

  4%|██▎                                                             | 739/19927 [10:10<4:24:21,  1.21it/s, loss=0.131]